<a href="https://colab.research.google.com/github/2303A51553/Natural-language-process/blob/main/project_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install gensim transformers tensorflow nltk scikit-learn rouge-score


import numpy as np
import pandas as pd
import re, string, torch
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense
from transformers import BertTokenizer, BertModel
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
